# Training Robust Neural Networks

Learn how to train neural networks in a robust way. In this chapter, you will use object-oriented programming to define PyTorch datasets and models and refresh your knowledge of training and evaluating neural networks. You will also get familiar with different optimizers and, finally, get to grips with various techniques that help mitigate the problems of unstable gradients so ubiquitous in neural nets training.

## PyTorch and object-oriented programming

### PyTorch Dataset

Time to refresh your PyTorch Datasets knowledge!

Before model training can commence, you need to load the data and pass it to the model in the right format. In PyTorch, this is handled by Datasets and DataLoaders. Let's start with building a PyTorch Dataset for our water potability data.

In this exercise, you will define a class called WaterDataset to load the data from a CSV file. To do this, you will need to implement the three methods which PyTorch expects a Dataset to have:

.__init__() to load the data,
.__len__() to return data size,
.__getitem()__ to extract features and label for a single sample.

In [67]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import torch
import torch.nn.init as init

Instructions:

- In the `.__init__()` method, load the data from csv_path to a pandas DataFrame and assign it to df.
- Convert df to a NumPy array and assign the result to self.data.
- Implement the `.__len__()` method to return the number of data samples.
- In the `.__getitem__()` method, get the label by slicing self.data to extract its last column for the index idx, similarly to how it's done for the features.

In [50]:
class WaterDataset(Dataset):
    def __init__(self, csv_path):
        super().__init__()
        # Load data to pandas DataFrame
        df = pd.read_csv(csv_path)
        # Convert data to a NumPy array and assign to self.data
        self.data = df.to_numpy(dtype='float32')

    # Implement __len__ to return the number of data samples
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        features = self.data[idx, :-1]
        # Assign last data column to label
        label = self.data[idx, -1]
        return features, label

## PyTorch DataLoader

The next step in preparing the training data is to set up a `DataLoader`. A PyTorch `DataLoader` can be created from a `Dataset` to load data, split it into batches, and perform transformations on the data if desired. Then, it yields a data sample ready for training.

Instructions:
- Create an instance of `WaterDataset` from `water_train.csv`, assigning it to `dataset_train`.
- Create `dataloader_train` based on `dataset_train`, using a batch size of two and shuffling the samples.
- Get a batch of features and labels from the DataLoader and print them.

In [51]:
# Create an instance of the WaterDataset
dataset_train = WaterDataset("datasets/water_potability/water_train.csv")

# Create a DataLoader based on dataset_train
dataloader_train = DataLoader(
    dataset_train,
    batch_size=2,
    shuffle=True,
)

# Get a batch of features and labels
features, labels = next(iter(dataloader_train))
print(features, labels)

tensor([[1.0000, 0.6651, 0.4217, 0.3616, 0.7161, 0.4697, 0.4536, 0.5569, 0.3411],
        [0.5241, 0.3169, 0.0685, 0.7824, 0.9872, 0.2791, 0.5302, 0.5734, 0.2569]]) tensor([0., 1.])


### PyTorch Model

You will use the OOP approach to define the model architecture. Recall that this requires setting up a model class and defining two methods inside it:

- `.__init__()`, in which you define the layers you want to use;

- `forward()`, in which you define what happens to the model inputs once it receives them; this is where you pass inputs through pre-defined layers.

Let's build a model with three linear layers and ReLU activations. After the last linear layer, you need a sigmoid activation instead, which is well-suited for binary classification tasks like our water potability prediction problem. Here's the model defined using `nn.Sequential()`, which you may be more familiar with:

``` 
net = nn.Sequential(
  nn.Linear(9, 16),
  nn.ReLU(),
  nn.Linear(16, 8),
  nn.ReLU(),
  nn.Linear(8, 1),
  nn.Sigmoid(),
)
```


Instructions:

- In the `.__init__()` method, define the three linear layers with dimensions corresponding to the model definition provided and assign them to `self.fc1`, `self.fc2`, and `self.fc3`, respectively.
- In the `forward()` method, pass the model input `x` through all the layers, remembering to add activations on top of them, similarly how it's already done for the first layer.

In [52]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Define the three linear layers
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        # Pass x through linear layers adding activations
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

## Optimizers, training, and evaluation

### Training loop

Time to refresh your knowledge on training loops! Let's train a classifier to predict water potability. You will use the model called `net`, which you built in the previous lesson.

In [ ]:
for epoch in range(1000):
    for features, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

### Optimizers

It's time to explore the different optimizers that you can use for training your model.

A custom function called `train_model(optimizer, net, num_epochs)` has been defined for you. It takes the optimizer, the model, and the number of epochs as inputs, runs the training loops, and prints the training loss at the end.

Let's use `train_model()` to run a few short trainings with different optimizers and compare the results!

In [58]:
criterion = nn.BCELoss()  # for binary classification


def train_model(optimizer: optim.Optimizer, net: Net, num_epochs: int):
    for epoch in range(num_epochs):
        total_loss = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels.view(-1, 1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
    print(f"Training loss after {num_epochs}: {total_loss / len(dataloader_train)}")

Instructions:
- Define the optimizer as Stochastic Gradient Descent.
- Define the optimizer as Root Mean Square Propagation (`RMSprop`), passing the model's parameters as its first argument.
- Define the optimizer as Adaptive Moments Estimation (`Adam`), setting the learning rate to `0.001`.

In [59]:
net = Net()

# Define the SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10: 0.6739581890542564


In [60]:
# Define the RMSprop optimizer
optimizer = optim.RMSprop(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10: 0.6708430322949387


In [65]:
# Define the Adam optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=10,
)

Training loss after 10: 0.6087973729092183


### Model evaluation

With the training loop sorted out, you have trained the model for 1000 epochs, and it is available to you as net. You have also set up a `test_dataloader` in exactly the same way as you did with `train_dataloader` before—just reading the data from the test rather than the train directory.

You can now evaluate the model on test data. To do this, you will need to write the evaluation loop to iterate over the batches of test data, get the model's predictions for each batch, and calculate the accuracy score for it. Let's do it!

Instructions:

- Set up the evaluation metric as `Accuracy` for binary classification and assign it to `acc`.
- For each batch of test data, get the model's outputs and assign them to `outputs`.
- After the loop, compute the total test accuracy and assign it to `test_accuracy`.

In [62]:
dataset_test = WaterDataset("datasets/water_potability/water_test.csv")

dataloader_test = DataLoader(
    dataset_test,
    batch_size=2,
    shuffle=True,
)

In [66]:
# Set up binary accuracy metric
acc = torchmetrics.Accuracy(task="binary")

net.eval()
with torch.no_grad():
    for features, labels in dataloader_test:
        # Get predicted probabilities for test data batch
        outputs = net(features)
        preds = (outputs >= 0.5).float()
        acc(preds, labels.view(-1, 1))

# Compute total test accuracy
test_accuracy = acc.compute()
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.6819085478782654


## Vanishing and exploding gradients

## Initialization and activation

The problems of unstable (vanishing or exploding) gradients are a challenge that often arises in training deep neural networks. In this and the following exercises, you will expand the model architecture that you built for the water potability classification task to make it more immune to those problems.

As a first step, you'll improve the weights initialization by using He (Kaiming) initialization strategy. To do so, you will need to call the proper initializer from the `torch.nn.init` module, which has been imported for you as `init`. Next, you will update the activations functions from the default ReLU to the often better ELU.

Instructions:

- Call the He (Kaiming) initializer on the weight attribute of the second layer, `fc2`, similarly to how it's done for `fc1`.
- Call the He (Kaiming) initializer on the weight attribute of the third layer, `fc3`, accounting for the different activation function used in the final layer.
- Update the activation functions in the `forward()` method from `relu` to `elu`.

In [68]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)

        # Apply He initialization
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid")

    def forward(self, x):
        # Update ReLU activation to ELU
        x = nn.functional.elu(self.fc1(x))
        x = nn.functional.elu(self.fc2(x))
        x = nn.functional.sigmoid(self.fc3(x))
        return x

### Batch Normalization

As a final improvement to the model architecture, let's add the batch normalization layer after each of the two linear layers. The batch norm trick tends to accelerate training convergence and protects the model from vanishing and exploding gradients issues.

Instructions:

- Add two `BatchNorm1d` layers assigning them to `self.bn1` and `self.bn2`
- In the `forward()` method, pass `x` through the second set of layers: the linear layer, the batch norm layer, and the activations, similarly to how it's done for the first set of layers.

In [69]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        # Add two batch normalization layers
        self.bn1 = nn.BatchNorm1d(16)
        self.bn2 = nn.BatchNorm1d(8)
        
        init.kaiming_uniform_(self.fc1.weight)
        init.kaiming_uniform_(self.fc2.weight)
        init.kaiming_uniform_(self.fc3.weight, nonlinearity="sigmoid") 
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = nn.functional.elu(x)

        # Pass x through the second set of layers
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.elu(x)

        x = nn.functional.sigmoid(self.fc3(x))
        return x

Which of the following statements is true about batch normalization?
- Adding batch normalization doesn't impact the number of parameters the model has to learn.
- Batch normalization normalizes a layer's inputs to a standard normal distribution and passes these normalized values further.
- Batch normalization effectively learns the optimal input distribution for each layer it precedes. (True)

By learning how to optimally re-scale the next layer's inputs, batch normalization mitigates the unstable gradients problems!